<a href="https://colab.research.google.com/github/ranjithgopalan/DataScience/blob/master/Covid19Analysis_Production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests 
import pandas as pd
import flask
import os
import time
import uuid
import numpy 
import json2html
%config IPCompleter.greedy=True
from pandas.io.json import json_normalize
from flask import Flask
from flask import render_template
from json2html import *
from flask import request

In [ ]:
#Convert to just country names
CountryResponse = requests.get("https://api.covid19api.com/countries")
if CountryResponse.status_code == 200:
    Countrydata = CountryResponse.json()

DataFrame_Country = pd.DataFrame([dict(Country=datum['Country'])
                   for datum in Countrydata ])



# update recovery and death rate 
covidSummaryResponse = requests.get("https://api.covid19api.com/summary")
if covidSummaryResponse.status_code == 200:
    data = covidSummaryResponse.json()
    DataFrame_CovidTotalSummary = pd.DataFrame(
    {"NewConfirmed":data['Global']['NewConfirmed'],"TotalConfirmed" : data['Global']['TotalConfirmed'],
         "NewDeaths": data['Global']['NewDeaths'], "NewRecovered":data['Global']['NewRecovered'],
     "TotalRecovered":data['Global']['TotalRecovered'],"RecoveredPercent":
     round((data['Global']['TotalRecovered']/data['Global']['TotalConfirmed'])*100,2),"ReportedDate": data['Date']},
    index = [1])
        
    DataFrame_CovidSummary = pd.DataFrame([dict(Country=datum['Country'],
                        NewConfirmed=datum['NewConfirmed'],
                        TotalConfirmed=datum['TotalConfirmed'],
                        NewDeaths=datum['NewDeaths'],
                        TotalDeaths=datum['TotalDeaths'],
                        NewRecovered=datum['NewRecovered'],
                        TotalRecovered=datum['TotalRecovered'],
                        Date= datum['Date']
                        )
                   for datum in data['Countries']])


    Recovered = [] 
    for TotalRecovered, TotalConfirmed in zip(DataFrame_CovidSummary['TotalRecovered'], DataFrame_CovidSummary['TotalConfirmed']):
        RecoveredPercent = (TotalRecovered/TotalConfirmed)*100
        Recovered.append(round(RecoveredPercent,2))

    DataFrame_CovidSummary["PercentOfRecovery"] = Recovered
    
    
    Death = [] 
    for TotalDeath, TotalConfirmed in zip(DataFrame_CovidSummary['TotalDeaths'], DataFrame_CovidSummary['TotalConfirmed']):
        RecoveredDeath = (TotalDeath/TotalConfirmed)*100
        Death.append(round(RecoveredDeath,2))

    DataFrame_CovidSummary["PercentOfDeath"] = Death
    


# Api for covide data for all countries
app = Flask(__name__)
@app.route('/CovideTotalSummary')
def index():
    return DataFrame_CovidSummary.sort_values(by=['TotalConfirmed'], ascending=False).to_json(orient='records')

# Api for covide data country wise
@app.route('/CovideSummary/Country', methods = ['GET'])
def index2():
    Country = request.args.get('Country')
    print(Country)
      
    return DataFrame_CovidSummary.loc[DataFrame_CovidSummary.Country == Country ].to_json(orient='records')


#Api for covide data  based on % of recovery
@app.route('/CovideSummary/Recovery', methods = ['GET'])
def index3():
    PercentOfRecovery = request.args.get('PercentOfRecovery')
    Mode = request.args.get('Mode')
    print(PercentOfRecovery)
    print(Mode)
    
    if Mode.lower() == 'greaterthan':
        return DataFrame_CovidSummary.loc[DataFrame_CovidSummary.PercentOfRecovery >= int(PercentOfRecovery) ].to_json(orient='records')
    elif Mode.lower() =='lessthan': 
        return DataFrame_CovidSummary.loc[DataFrame_CovidSummary.PercentOfRecovery <= int(PercentOfRecovery) ].to_json(orient='records')
    
#Api for covide data  based on % of death
@app.route('/CovideSummary/Death', methods = ['GET'])
def index4():
    PercentOfDeath = request.args.get('PercentOfDeath')
    Mode = request.args.get('Mode')
    print(PercentOfDeath)
    print(Mode)
    
    if Mode.lower() == 'greaterthan':
        return DataFrame_CovidSummary.loc[DataFrame_CovidSummary.PercentOfDeath >= int(PercentOfDeath) ].to_json(orient='records')
    elif Mode.lower() =='lessthan': 
        return DataFrame_CovidSummary.loc[DataFrame_CovidSummary.PercentOfDeath <= int(PercentOfDeath) ].to_json(orient='records')


#Api for list of countries   
@app.route('/CovideSummary/ListOfCountries')
def index5():
    return DataFrame_Country.to_json(orient='values')


# Api for top affected countries
@app.route('/CovideSummary/Top_affectedCountries', methods = ['GET'])
def index6():
    TopCount = request.args.get('Top')
    DataFrame_CovidSummaryTop_Countries = DataFrame_CovidSummary[['Country','TotalConfirmed', 'TotalDeaths']].sort_values(by=['TotalConfirmed'], ascending=False )
    return DataFrame_CovidSummaryTop_Countries[:int(TopCount)].to_json(orient='records')


    
if __name__ == '__main__':
    app.run(host='127.0.0.1', port='8050', debug=False)


NameError: name 'Countrydata' is not defined